In [6]:
import json
import csv
import os

# ------------------- 파일 경로 -------------------
# 데이터 경로
TRAIN_JSON_PATH = r"..\cadenza_data_train\metadata\train_metadata.json"
VALID_JSON_PATH = r"..\cadenza_data_valid\metadata\valid_metadata.json"
EVAL_JSON_PATH = r"..\cadenza_data_eval\metadata\eval_metadata.json"
JSON_PATH = VALID_JSON_PATH # 사용할 JSON 선택

# CSV 저장 경로
TRAIN_CSV_PATH = r"..\cadenza_data_train\metadata\train_metadata_l_label.csv"
VALID_CSV_PATH = r"..\cadenza_data_valid\metadata\valid_metadata_l_label.csv"
EVAL_CSV_PATH = r"..\cadenza_data_eval\metadata\eval_metadata_l_label.csv"
CSV_PATH = VALID_CSV_PATH # 저장할 CSV 선택


# ------------------- 단어 처리 -------------------
# 두 단어로 세야 하는 단어들
DOUBLE_WORDS = [
    "wanna", "wana", "gonna", "gotta", "gimme", "tv", "dj", "tryna", "sweetle", "sorta",
    "haiku-ing", "she's", "shes", "hes", "you're", "you\u2019re", "youre", "we're", "i'm", "im",
    "it's", "that's", "he's", "there's", "what's", "they're", "who's", "'tis", "tis",
    "i'll", "they'll", "you'll", "we'll", "he'll",
    "i've", "ive", "you've", "we've", "could've", "they've", "where've", "should've",
    "don't", "dont", "wasn't", "isnt", "didn't", "won't", "didnt", "aren't", "wouldn't",
    "weren't", "doesn't", "couldn't", "haven't", "shouldn't", "isn't", "mustn't",
    "who'd", "we'd", "i'd", "you'd", "youd", "she'd", "he'd", "why'd", "how'd", "know'd",
    "y'all", "let's",
]

# 특수 예외 단어: word → 실제 단어 개수
EXCEPTION_WORDS = {
    "i'ma": 4,
    "haiku-ing": 3,
    "it\u2019s": 1,  # JSON에서 유니코드 아포스트로피 쓴 경우만 1로 처리
}

# 반드시 분리해야 할 복합어들 (변환 규칙)
SPLIT_WORDS = {
    "junkfood": ["junk", "food"],
    "riverwest": ["river", "west"],
    "pariassound": ["parias", "sound"],
    "eachother": ["each", "other"],
    "chickenfried": ["chicken", "fried"],
    "shotglass": ["shot", "glass"],
    "oneway": ["one", "way"],
    "motorlaunch": ["motor", "launch"],
    "onehundred": ["one", "hundred"],
    "twentyten": ["twenty", "ten"],
    "rockstar": ["rock", "star"],
    "sugartown": ["sugar", "town"],
    "uptime": ["up", "time"],
    "reclasp": ["re", "clasp"],
    "sixtyfive": ["sixty", "five"],
}

DOUBLE_WORDS = []
EXCEPTION_WORDS = {}
SPLIT_WORDS = {}

### check prompt n_words

In [7]:
def process_prompt(prompt):
    """
    프롬프트를 처리해서
    1) 클린된 prompt (split_words 반영)
    2) 단어 리스트와 각 단어 word_count
    3) 총 단어 수 (count)
    반환
    """
    # ① 특수문자 제거
    prompt_no_special = prompt.replace("*", "").replace(",", "")
    words = []
    total_count = 0
    prompt_words_for_csv = []

    for w in prompt_no_special.split():
        w_lower = w.lower()

        # ① 예외 단어
        if w_lower in EXCEPTION_WORDS:
            total_count += EXCEPTION_WORDS[w_lower]
            w = w.replace("\u2019", "'").lower()
            words.append((w, EXCEPTION_WORDS[w_lower]))
            prompt_words_for_csv.append(w)
            continue

        # ② 반드시 분리할 복합어
        w_norm = w.replace("\u2019", "'").lower()
        if w_norm in SPLIT_WORDS:
            for sub in SPLIT_WORDS[w_norm]:
                words.append((sub, 1))
                total_count += 1
                prompt_words_for_csv.append(sub)  # CSV용
            continue

        # ③ double_words
        if w_lower in DOUBLE_WORDS:
            words.append((w, 2))
            total_count += 2
            prompt_words_for_csv.append(w)
        else:
            words.append((w, 1))
            total_count += 1
            prompt_words_for_csv.append(w)

    # prompt_clean: split_words 반영 후 띄어쓰기 단위로 연결
    prompt_clean = " ".join(prompt_words_for_csv)
    return prompt_clean, words, total_count


In [8]:
# =================== 실행 ===================
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

mismatch_cases = []
for entry in data:
    # process_prompt 사용
    _, _, prompt_word_count = process_prompt(entry["prompt"])
    
    if prompt_word_count != entry["n_words"]:
        mismatch_cases.append({
            "signal": entry["signal"],
            "prompt": entry["prompt"],
            "n_words": entry["n_words"],
            "prompt_word_count": prompt_word_count
        })

print(f"총 불일치 케이스 수: {len(mismatch_cases)}\n")
print("최대 20개 케이스 예시:")
for case in mismatch_cases[:20]:
    print(case)

총 불일치 케이스 수: 0

최대 20개 케이스 예시:


### make csv

In [9]:
def save_to_csv(data, csv_path, is_train=True):
    """CSV 저장"""
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        # 헤더
        if is_train:
            header = [
                "signal", "prompt", "response", "n_words",
                "words_correct", "correctness", "hearing_loss",
                "word", "word_count"
            ]
        else:
            # header = [
            #     "signal", "prompt", "n_words", "hearing_loss",
            #     "word", "word_count"
            # ]
            header = [
                "signal", "prompt", "response", "n_words",
                "words_correct", "correctness", "hearing_loss",
                "word", "word_count"
            ]
        writer.writerow(header)

        for entry in data:
            prompt_clean, words, _ = process_prompt(entry["prompt"])  # process_prompt 사용

            for w, wc in words:
                word_clean = w.replace("\u2019", "'")  # CSV에 ’ → ' 변환

                if is_train:
                    writer.writerow([
                        entry["signal"], prompt_clean, entry["response"],
                        entry["n_words"], entry["words_correct"], entry["correctness"],
                        entry["hearing_loss"], word_clean, wc
                    ])
                else:
                    writer.writerow([
                        entry["signal"], prompt_clean,
                        entry["n_words"], entry["hearing_loss"],
                        word_clean, wc
                    ])


# =================== 실행 예시 ===================
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

is_train = "response" in data[0]
csv_path = os.path.splitext(JSON_PATH)[0] + ".csv"

save_to_csv(data, CSV_PATH, is_train)
print(f"CSV 저장 완료 → {CSV_PATH}")


CSV 저장 완료 → ..\cadenza_data_valid\metadata\valid_metadata_l_label.csv


In [10]:
import pandas as pd

# CSV 로드
df = pd.read_csv(CSV_PATH)

# signal별 word_count 합계 계산
word_count_sum = df.groupby("signal")["word_count"].sum().reset_index()
word_count_sum = word_count_sum.rename(columns={"word_count": "word_count_sum"})

# 원본 n_words 정보 가져오기 (train/valid 공통)
n_words_df = df.drop_duplicates("signal")[["signal", "n_words"]]

# 합치기
check_df = pd.merge(word_count_sum, n_words_df, on="signal")
check_df["match"] = check_df["word_count_sum"] == check_df["n_words"]

# 불일치만 출력
mismatch = check_df[~check_df["match"]]
print(f"총 불일치 signal 수: {len(mismatch)}")
print(mismatch.head(20))  # 최대 20개 예시

총 불일치 signal 수: 0
Empty DataFrame
Columns: [signal, word_count_sum, n_words, match]
Index: []
